In [1]:
from bs4 import BeautifulSoup
import random
import html
import requests
import re
from time import sleep
from urllib.parse import quote
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

In [13]:
site = 'https://myshows.me/community/users/?page={}'
max_num_page = 5000

In [28]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)
    
def extract_users_from_page(site_num):
    soup = BeautifulSoup(requests.get(site.format(site_num)).text, 'html.parser')
    res = []
    
    links = soup.find_all('a', class_='userBlock linkBlock')
    for link in links:
        res.append(link.attrs['href'])
    
    return res

def extract_grades_from_user_page(user_url):
    soup = BeautifulSoup(requests.get(user_url).text, 'html.parser')
    res = {}
    
    tabs = soup.find_all('div', class_='tabs_cont')
    
    for completed_tab in tabs:
        all_grades = completed_tab.find_all('tr')
        for grade in all_grades[:-1]:
            grade_parts = grade.find_all('td')
            id_s = grade_parts[0].find('a').attrs['href'].split('/')[-2]
            grade_value = grade_parts[1].find('span').attrs['class'][1][1]
            res[id_s] = grade_value
    
    return res

In [15]:
%%time


with ProcessPoolExecutor(max_workers=5) as executor:
    result = list(executor.map(extract_users_from_page,
                               list(range(max_num_page))))
    
unpack_result = []
for l in result:
    unpack_result += l

with open('user_urls.txt', 'w') as fin:
    for row in unpack_result:
        fin.write(row + '\n')

CPU times: user 6.7 s, sys: 1.15 s, total: 7.85 s
Wall time: 22min 40s


In [18]:
print(len(set(unpack_result)), len(unpack_result))
user_urls = list(set(unpack_result))

149981 150000


In [34]:
%%time

# try:
extract_grades_from_user_page('https://myshows.me/vk819107')
# except:
#     pass

CPU times: user 295 ms, sys: 7.41 ms, total: 302 ms
Wall time: 478 ms


{'16933': '0', '51398': '0', '28600': '0'}

In [35]:
def ignore_errors_extract_grades_from_user_page(user_url):
    try:
        return extract_grades_from_user_page(user_url)
    except:
        print('parsing {} finish with error'.format(user_url))

In [44]:
%%time

with ProcessPoolExecutor(max_workers=5) as executor:
    result = list(executor.map(ignore_errors_extract_grades_from_user_page,
                               list(user_urls)))

user_ratings = pd.DataFrame(columns=['user_id', 'show_id', 'grade'])

for i, grades in enumerate(result):
    for show_id, grade in grades.items():
        user_ratings = user_ratings.append({'user_id' : i,
                                           'show_id' : show_id,
                                           'grade' : grade}, ignore_index=True)
user_ratings.to_csv('user_ratings.csv')

parsing https://myshows.me/id172969629 finish with error
parsing https://myshows.me/Starky finish with error
parsing https://myshows.me/berendyaev finish with error
parsing https://myshows.me/Aliynchik finish with error
parsing https://myshows.me/Yuliya finish with error


KeyboardInterrupt: 

In [43]:
user_ratings.head()

,user_id,show_id,grade
0,0,55672,5
1,0,22410,5
2,0,25119,5
3,0,34265,5
4,0,54291,5
